In [6]:
import json
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso,ElasticNet
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [7]:
import json
with open("algoparams_from_ui.json.rtf",'r') as file:
    f=file.read()

In [ ]:
!pip install striprtf

In [8]:
from striprtf.striprtf import rtf_to_text

In [9]:
text = rtf_to_text(f)

In [10]:
json_object=json.loads(text)

1) Read the target and type of regression to be run

In [11]:
json_object['design_state_data']['target']

{'prediction_type': 'Regression',
 'target': 'petal_width',
 'type': 'regression',
 'partitioning': True}

In [12]:
for features,values in json_object['design_state_data']['feature_handling'].items():
    print(features)
    print(values)

sepal_length
{'feature_name': 'sepal_length', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values', 'impute_value': 0}}
sepal_width
{'feature_name': 'sepal_width', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'custom', 'impute_value': -1}}
petal_length
{'feature_name': 'petal_length', 'is_selected': True, 'feature_variable_type': 'numerical', 'feature_details': {'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values', 'impute_value': 0}}
petal_width
{'feature_name': 'peta

In [13]:
import pandas as pd
data=pd.read_csv('iris.csv')

# Feature Handling

2) Read the features (which are column names in the csv) and figure out what missing imputation needs to be applied and apply that to the columns loaded in a dataframe

In [14]:
def impute(Data,feature,feature_variable_type,feature_details):
    if feature_variable_type=="numerical":
        if feature_details['impute_with']=='Average of values':
            Data[feature].fillna(feature_details['impute_value'],inplace=True)
        if feature_details['impute_with']=='custom':
            Data[feature].fillna(feature_details['impute_value'],inplace=True)
    return Data    

In [15]:
def feature_handling(json_object,data):
    for feature,values in json_object['design_state_data']['feature_handling'].items():
        Data=data
        if values['is_selected']==True:
            data=impute(Data,feature,values['feature_variable_type'],values['feature_details'])  
    return Data

In [16]:
data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [17]:
df=feature_handling(json_object,data)

# Train Test Split

In [18]:
x=df.iloc[:,:-2]
y=df[json_object['design_state_data']['target']['target']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
x_train

,sepal_length,sepal_width,petal_length
15,5.7,4.4,1.5
125,7.2,3.2,6.0
11,4.8,3.4,1.6
127,6.1,3.0,4.9
51,6.4,3.2,4.5
...,...,...,...
71,6.1,2.8,4.0
106,4.9,2.5,4.5
14,5.8,4.0,1.2
92,5.8,2.6,4.0


In [21]:
y_train

15     0.4
125    1.8
11     0.2
127    1.8
51     1.5
      ... 
71     1.3
106    1.7
14     0.2
92     1.2
102    2.1
Name: petal_width, Length: 120, dtype: float64

# Feature Reduction

3) Compute feature reduction based on input. See the screenshot below where there can be No Reduction, Corr with Target, Tree-based, PCA. Please make sure you write code so that all options can work. If we rerun your code with a different Json it should work if we switch No Reduction to say PCA.

In [22]:
def feature_reduction(json_object,x,y):
    dict_reduction=json_object['design_state_data']['feature_reduction']
    if dict_reduction['No_reduction']['is_selected']==True:
        x=x
        
        
    if dict_reduction['correlation_with_target']['is_selected']==True:
        print(x)#write the code here
        
    
    if dict_reduction['Tree-based']['is_selected']==True:
        clf=RandomForestClassifier(n_estimators=4)
        clf = clf.fit(x, y)
        model = SelectFromModel(clf, prefit=True)
        x= model.transform(x)
   
    if dict_reduction['Principal Component Analysis']['is_selected']==True:
        pca=PCA(n_components=dict_1['Principal Component Analysis']['num_of_features_to_keep'])
        pca.fit(X_features)
        x = pca.transform(x)
    
    return x       

In [23]:
dict_model=json_object['design_state_data']['algorithms']
for i in dict_model:
    print(i)

RandomForestClassifier
RandomForestRegressor
GBTClassifier
GBTRegressor
LinearRegression
LogisticRegression
RidgeRegression
LassoRegression
ElasticNetRegression
xg_boost
DecisionTreeRegressor
DecisionTreeClassifier
SVM
SGD
KNN
extra_random_trees
neural_network


4) Parse the Json and make the model objects (using sklean) that can handle what is required in the “prediction_type” specified in the JSON (See #1 where “prediction_type” is specified). Keep in mind not to pick models that don’t apply for the prediction_type specified

In [24]:
def model(json_object,cv,x,y):
    dict_model=json_object['design_state_data']['algorithms']
    if json_object['design_state_data']['target']['prediction_type']=='Regression':
        if dict_model['RandomForestRegressor']['is_selected']:
            model=RandomForestRegressor()
            param_RF = {'n_estimators': [10, 20, 30],  # Number of trees in the forest
                         'max_depth': [20, 25, 30],    # Maximum depth of the trees
                         'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
                         'min_samples_leaf': [5, 10, 20],  # Minimum number of samples required to be at a leaf node
                         'bootstrap': [True, False]  # Whether bootstrap samples are used 
                       }
            best_parameter,best_model=model_fit(model,param_RF,cv,x,y)
            
        
        if dict_model['LinearRegression']['is_selected']:
            model=LinearRegression()
            param_linear = {'fit_intercept': [True, False],  # Whether to calculate the intercept
                            'normalize': [True, False],  # Whether to normalize the regressors before fitting
                            'copy_X': [True, False],  # Whether to copy the data (if False, may overwrite data)
                           }
            best_parameter,best_model=model_fit(model,param_linear,cv,x,y)
           
            
            
        if dict_model['GBTRegressor']['is_selected']:
            model=GradientBoostingRegressor()
            param_GBT= {'n_estimators': [67, 89],  # Number of boosting stages
                        'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
                        'loss': ['ls', 'lad', 'huber', 'quantile'],  # Loss function
                        'subsample': [1.0, 0.9, 0.8],  # Fraction of samples used for fitting the trees
                        'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
                        'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
                        'max_depth': [5, 7, 10]  # Maximum depth of the individual trees
                       }
            best_parameter,best_model=model_fit(model,param_GBT,cv,x,y)
            
           
            
        
        if dict_model['RidgeRegression']['is_selected']:
            model=Ridge()
            param_ridge = {'alpha': [0.1, 0.5, 1.0, 2.0],  # Regularization strength (alpha)
                           'fit_intercept': [True, False],  # Whether to calculate the intercept
                           'normalize': [True, False],  # Whether to normalize the regressors before fitting
                           'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],  # Solver for optimization
                           'max_iter': [10, 30, 50],  # Maximum number of iterations
                           }
            best_parameter,best_model=model_fit(model,param_Ridge,cv,x,y)
        
        if dict_model['LassoRegression']['is_selected']:
            model= Lasso()
            param_Lasso = {'alpha': [0.1, 0.5, 1.0, 2.0],  # Regularization strength (alpha)
                           'fit_intercept': [True, False],  # Whether to calculate the intercept
                           'normalize': [True, False],  # Whether to normalize the regressors before fitting
                           'max_iter': [10, 30, 50],  # Maximum number of iterations
                           }
            best_parameter,best_model=model_fit(model,param_Lasso,cv,x,y)
            
            
        if dict_model['ElasticNetRegression']['is_selected']:
            model=ElasticNet()
            param_Elastic = {'alpha': [0.1, 0.5, 1.0, 2.0],  # Regularization strength (alpha)
                             'l1_ratio': [0.1, 0.5, 0.7, 0.9],  # Mixing parameter for L1 and L2 penalties
                             'fit_intercept': [True, False],  # Whether to calculate the intercept
                             'normalize': [True, False],  # Whether to normalize the regressors before fitting
                             'max_iter': [10, 30, 50],  # Maximum number of iterations
                             }
            best_parameter,best_model=model_fit(model,param_Elastic,cv,x,y)
            
        
        if dict_model['DecisionTreeRegressor']['is_selected']:
            model = DecisionTreeRegressor()
            param_DT = {'max_depth': [4, 5, 6, 7],  # Maximum depth of the tree
                        'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
                        'min_samples_leaf': [12, 6],  # Minimum number of samples required to be at a leaf node
                        'criterion': ['mse'],  # Splitting criterion (Mean Squared Error)
                        'splitter': ['best', 'random'],  # Strategy to choose the split at each node
                        }
            best_parameter,best_model=model_fit(model,param_DT,cv,x,y)
        
        if dict_model['SVM']['is_selected']:
            model = SVR()
            param_SVR = {'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernels to consider
                         'C': [566, 79],  # Regularization parameter (C)
                         'gamma': ['scale', 'auto'],  # Kernel coefficient (gamma)
                         'tol': [7],  # Tolerance for stopping criterion
                         'max_iter': [7]  # Maximum number of iterations for solver
                         }
            best_parameter,best_model=model_fit(model,param_SVR,cv,x,y)
        
        if dict_model['KNN']['is_selected']:
            model = KNeighborsRegressor()
            param_knn = {'n_neighbors': [25,50,75,100],  # Number of neighbors to use
                         'weights': ['distance'],  # Weighting method for neighbors (distance-based)
                         'algorithm': ['auto'],  # Algorithm used to compute nearest neighbors
                         'p': [2],  # Power parameter for Minkowski distance (Euclidean distance)
                        }
            best_parameter,best_model=model_fit(model,param_SVR,cv,x,y)
        #if dict_model['neural_network']['is_selected']:
            #model = MLPRegressor()
            
    return  best_parameter,best_model        

5) Run the fit and predict on each model – keep in mind that you need to do hyper parameter tuning i.e., use GridSearchCV

In [25]:
def model_fit(model,param,cv,x,y):
    grid_search = GridSearchCV(estimator=model, param_grid=param, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(x,y)
    best_parameter = grid_search.best_params_
    best_model = grid_search.best_estimator_
    
    return best_parameter,best_model

In [26]:
num_folds=6
cv =TimeSeriesSplit(n_splits=num_folds)
_,best_model=model(json_object,cv,x_train,y_train)

In [27]:
def model_predict(best_model,x_test):
    y_pred = best_model.predict(x_test)
    return y_pred

In [28]:
y_pred=model_predict(best_model,x_test)

6) Log to the console the standard model metrics that apply

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [30]:
def performance_metric(y_pred,y_test):
    MSE=mean_squared_error(y_test,y_pred)
    MAE=mean_absolute_error(y_test,y_pred)
    
    return MSE,MAE,r2_score

In [31]:
MSE,MAE,r2_score=performance_metric(y_pred,y_test)   

In [32]:
print("Mean_squared_error",MSE)
print("Mean_absolute_error",MAE)


Mean_squared_error 0.04942055410686957
Mean_absolute_error 0.16715074229905563
